![bse_logo_textminingcourse](https://bse.eu/sites/default/files/bse_logo_small.png)

# Machine Learning for Finance - Assignment 2

### by Amber Walker and Clarice Mottet
### All work was distributed and completed equally.

0. **[Part 0: Set Up and Import](#part0)**
- **Objective**: Initialize programming environment and data.
- **Tasks:**
  - Initialize libraries.

1. **[Part 1: Backtesting trading strategies based on sentiment indicators.](#part1)**
- **Objective**: 
- **Tasks:**
    - tasks

2. **[Part 2: Factor Models and Sentiment](#part2)**
- **Objective**: Compare differences/similarities between factor models.
- **Tasks:**
    - Choose 9 stocks from the dataset dataset.rds using two consecutive years of data for experiments 2016-06 to 2018-06 (chosen and registered in Prob. 2 of sheet HW2 on the drive)
    - Compute the Robust (ellipsoid) Global Maximum Return Portfolio using as perturbation matrix Sigma (S) corresponding to the factor models:
      1. The Fama-French 3-factors returns 3FF
      2. The Sentiment indicator PNlog factor model
    - Try different kappas (at least 3 values in (0,1)) and multiple robust noisy solutions to check for sensitivity. Comment on the differences/similarities of results for both cases of Sigma.


## <a id='part0'>Part 0: Set Up</a>


In [21]:
#libraries

import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cvxpy as cp
from scipy.linalg import cholesky
import scipy.linalg as la
import altair as alt
import yfinance as yfin  #These need lxml and this is deprecated
import pickle
import seaborn as sns

from numpy.linalg import solve, norm
from scipy.cluster.hierarchy import fcluster
from statsmodels.stats.correlation_tools import cov_nearest

current_directory = os.getcwd()

In [22]:
def log_return(df):
    return np.log(df).diff().drop(index=df.index[0], axis=0, inplace=False)

def na_fill(df):
    # Simulates `na.fill(as.xts(rowAvgs(df),order.by=index(df)),"extend")`
    return df.interpolate(method="linear").fillna(method="bfill")

def create_index(df):
    # the linear method already does ffill for the tail values
    return na_fill(df.mean(axis=1, skipna=False))

def cov2cor(df):
    ind = df.index
    col = df.columns
    sigma = np.sqrt(np.diag(df))
    return df.div(pd.DataFrame(np.outer(sigma, sigma), index=ind, columns=col))

def clusterplot(cov, ncluster=6):
    sns.set(style='white')
    sns_plot = sns.clustermap(cov2cor(cov), cmap='vlag_r', robust=True, method="complete", vmin=-1, vmax=1)
    plt.setp(sns_plot.ax_heatmap.yaxis.get_majorticklabels(), color='red', fontSize=14)
    plt.setp(sns_plot.ax_heatmap.xaxis.get_majorticklabels(), color='red', fontSize=14)
    plt.title("Correlation")

    Z = sns_plot.dendrogram_row.linkage
    clusters = list(fcluster(Z, ncluster, criterion='maxclust'))
    assign_cluster = dict((k, clusters[k]) for k in range(len(cov.columns)))
    assigned_idx = [assign_cluster[idx] for idx in sns_plot.dendrogram_row.reordered_ind]
    i = 0
    while i < len(assigned_idx):
        cluster = assigned_idx[i]
        counter = 0
        while i < len(assigned_idx) and assigned_idx[i] == cluster:
            counter += 1
            i += 1
        sns_plot.ax_heatmap.add_patch(plt.Rectangle(
            xy=(i-counter, i-counter),
            width=counter,
            height=counter,
            fill=None, color='black', lw=1.5
        ))
    plt.show()

## <a id='part2'>Part 2: Factor Models and Sentiment</a>
- **Objective**: Compare differences/similarities between factor models
- **Tasks:**
    - Choose 9 stocks from the dataset dataset.rds using two consecutive years of data for experiments 2016-06 to 2018-06 (chosen and registered in Prob. 2 of sheet HW2 on the drive)
    - Create function code for the Robust (ellipsoid) Global Maximum Return Portfolio using as perturbation matrix Sigma (S) corresponding to the factor models:
      1. The Fama-French 3-factors returns 3FF
      2. The Sentiment indicator PNlog factor model
    - Try different kappas (at least 3 values in (0,1)) and multiple robust noisy solutions to check for sensitivity using the two different factor models. Comment on the differences/similarities of results for both cases of Sigma.

- Choose 9 stocks from the dataset dataset.rds using two consecutive years of data for experiments 2016-06 to 2018-06 (chosen and registered in Prob. 2 of sheet HW2 on the drive)

In [23]:
#-----------------------------------------
#Import in Stocks

dataset = pd.read_pickle(f'{current_directory}/Inputs/dataset.pkl')

#Subset to 9 stocks
#Subset to time frame 2016-06 to 2018-06

keep_stocks = ['AAPL', 'ABBV', 'AMZN', 'DIS', 'HSBC', 'JPM', 'MCD', 'MSFT', 'XOM']

# print(dataset['BBr'].columns)

for key in dataset.keys():
    dataset[key] = dataset[key][keep_stocks]
    dataset[key] = dataset[key][(dataset[key].index >= pd.to_datetime('2016-06-01'))&(dataset[key].index < pd.to_datetime('2018-06-01'))]

#-----------------------------------------
#Import in Fama-French factors

fama_lib = pd.read_csv(f'{current_directory}/Inputs/F-F_Research_Data_Factors_daily.CSV', skiprows=4, skipfooter=1, index_col=0, engine='python')
fama_lib.index = pd.to_datetime(fama_lib.index, format="%Y%m%d")
fama_lib = fama_lib.iloc[:, 0:3] # We only use the first 3 columns

#-----------------------------------------
# Prepare stock data

stockPrices = dataset["adjusted"]
X = log_return(stockPrices)
T, N = X.shape  # nrow and ncol

F_FamaFrench = fama_lib.loc[X.index]/100   # Fama-French factors
BBrMkt = create_index(dataset["BBr"])/100  # BBr Market index
PNlogMkt = create_index(dataset["PNlog"])  # PNlog Market index
SentIndx = PNlogMkt.loc[X.index]           # Sentiment index

# split data into training and test data
T_trn = round(0.5*T)
X_trn = X.iloc[0:T_trn, ]
X_tst = X.iloc[T_trn:T, ]

F_FamaFrench_trn = F_FamaFrench.iloc[0:T_trn, ]
F_FamaFrench_tst = F_FamaFrench.iloc[T_trn:T, ]

SentIndx_trn = SentIndx.iloc[0:T_trn,]
SentIndx_tst = SentIndx.iloc[T_trn:T,]


- Create function code for the Robust (ellipsoid) Global Maximum Return Portfolio using as perturbation matrix Sigma (S) corresponding to the factor models:
    1. The Fama-French 3-factors returns 3FF
    2. The Sentiment indicator PNlog factor model

In [24]:
#Functions

def portfolioMaxReturnRobustEllipsoid(mu_hat, S, kappa, color, note_method):
    mu_hat = np.array(mu_hat)
    S12 = la.cholesky(S)  # S12.T @ S12 = Sigma
    w = cp.Variable(len(mu_hat))
    prob = cp.Problem(cp.Maximize(mu_hat @ w - kappa * cp.norm(S12 @ w, p=2)), constraints=[w >= 0, sum(w) == 1])
    result = prob.solve()

    w_GMRP_robust_df = pd.DataFrame(w.value)
    w_GMRP_robust_df['stock'] = keep_stocks
    w_GMRP_robust_df.columns = ['w', 'stock']

    chart = alt.Chart(w_GMRP_robust_df).mark_bar(size=12, color=color).encode(
        x='stock:N',
        y='w:Q'
    ).properties(width=125, title=f'Portfolilo Weight: Factor-{note_method}, kappa-{kappa}')
    # chart.show()

    return w.value, chart

def fama_french_3F(X_trn):
    F = F_FamaFrench_trn.copy()
    F.insert(0, 'ones', 1)
    Gamma = pd.DataFrame(solve(F.T @ F, F.T.to_numpy() @ X_trn.to_numpy()).T, columns=["alpha", "beta1", "beta2", "beta3"])
    alpha = Gamma["alpha"]
    B = Gamma[["beta1", "beta2", "beta3"]]
    E = pd.DataFrame((X_trn.T - Gamma.to_numpy() @ F.to_numpy().T).T, index=X_trn.index)
    PsiFF = (E.T @ E) / (T_trn - 4)
    Sigma_FamaFrench = B.to_numpy() @ F_FamaFrench_trn.cov() @ B.to_numpy().T + np.diag(np.diag(PsiFF))
    return Sigma_FamaFrench

def sentiment_ind(X_trn):
    F = SentIndx_trn.copy().to_frame()
    F.insert(0, 'ones', 1)
    Gamma = pd.DataFrame(solve(F.T @ F, F.T.to_numpy() @ X_trn.to_numpy()).T, columns=["alpha", "beta"])
    alpha = Gamma["alpha"]
    beta = Gamma["beta"]
    E = pd.DataFrame((X_trn.T - Gamma.to_numpy() @ F.to_numpy().T).T, index=X_trn.index)
    Psi_Sent = (E.T @ E) / (T_trn - 2)
    Sigma_SentInx = SentIndx_trn.var() * np.outer(beta, beta) + np.diag(np.diag(Psi_Sent))
    return Sigma_SentInx

def is_pos_def(x, epsilon):
    return np.all(np.linalg.eigvals(x) > epsilon)

def add_noise(mu, Sigma, w_GMRP_robust, kappa, color, note_method):

    w_all_GMRP_robust_ellipsoid = np.expand_dims(w_GMRP_robust, axis=1)

    #multiple robust solutions
    np.random.seed(100)
    for i in range(6):
        X_noisy = pd.DataFrame(np.random.multivariate_normal(mu, Sigma, size=T))
        mu_noisy = X_noisy.mean(axis=0)
        Sigma_noisy = X_noisy.cov()
        # Sigma_noisy = factor_method(X_noisy)
        if not is_pos_def(Sigma_noisy, 0.001):
            Sigma_noisy = cov_nearest(Sigma_noisy)
        w_GMRP_robust_ellipsoid_noisy, chart = portfolioMaxReturnRobustEllipsoid(mu_noisy, Sigma_noisy, kappa, color, note_method)
        w_all_GMRP_robust_ellipsoid = np.concatenate((w_all_GMRP_robust_ellipsoid,
                                                    np.expand_dims(w_GMRP_robust_ellipsoid_noisy, axis=1)), axis=1)

    w_all_GMRP_robust_ellipsoid = pd.DataFrame(w_all_GMRP_robust_ellipsoid)
    w_all_GMRP_robust_ellipsoid["stock"] = keep_stocks

    long_w_all_GMRP_robust_ellipsoid = pd.melt(
        w_all_GMRP_robust_ellipsoid,
        id_vars=['stock'],
        var_name='run',
        value_name='weight')

    chart = alt.Chart(long_w_all_GMRP_robust_ellipsoid).mark_bar(size=10, color=color).encode(
        x=alt.X('run:O', axis=None),
        y='weight:Q',
        column=alt.Column('stock:N', spacing=2)
    ).properties(width=125, title=f'Portfolilo Weight with Noise: Factor-{note_method}, kappa-{kappa}')

    chart.show()

    return


In [25]:
Sigma_true = X_tst.cov()

ntype = "fro"  # Frobenious
error = pd.DataFrame({
    "SCM": norm(X_trn.cov().to_numpy() - Sigma_true.to_numpy(), ord=ntype),
    "FamaFrench": norm(fama_french_3F(X_trn) - Sigma_true.to_numpy(), ord=ntype),
    "SentInd": norm(sentiment_ind(X_trn) - Sigma_true.to_numpy(), ord=ntype)
}, index=[0])

order = ["SCM", "FamaFrench", "SentInd"]
alt.Chart(error.melt(value_name='error')).mark_bar().encode(
    x = alt.X('variable', title='', axis=alt.Axis(labelAngle=0), sort=order),
    y = 'error',
    color = alt.value("mediumaquamarine")
).properties(
    title="Error in Estimation of Covariance Matrix",
    height=300,
    width=400
)


alt.Chart(...)

- Try different kappas (at least 3 values in (0,1)) and multiple robust noisy solutions to check for sensitivity using the two different factor models. Comment on the differences/similarities of results for both cases of Sigma.

For our analysis we've chosen our $\kappa$ values to be 0.1 representing a very risk-averse return estimate, 0.7 a less risk-averse but still moderate and 0.9 representing a very risk-tolerate return estimate.

We see for the lower $\kappa$ value at 0.1, the best stock to maximize portfolio returns in a conservative approach is AMZN, APPL, MKST. We see for the risk-tolerate moderate and aggressive $\kappa$ values give similar stock suggestions of DIS and MCD. Even with noise incorporated, we see similar stock suggestions to maximize returns respective to $\kappa$.

For low-risk $\kappa$ value 0.1 we see AMZN, MKST, APPL as the suggestions for maximizing returns, for the more risk and moderate and high-risk $\kappa$ values we see DIS, MCD and MKST. With noise, we see for the low-risk $\kappa$ that MKST emerges ahead of AMZN as the stock to best maximize returns and for the other $\kappa$ values we see MCD and DIS as the leads and not so much MKST.

In comparison of the two factor models, we see similar results for the $\kappa \text{=0.1}$ but for the higher $\kappa$ values the difference really emerges with regard to MKST. The Fama-French-3F factor model did not indicate MKST for a higher risk approach and while the Sentiment Indicator factor model did. What is interesting is that with noise included MSKT no longer ends up as a suggestion to maximize returns.

We don't see a big difference in results without or with noise using the Fama-French-3F factor model but we do see differences in results between the not noisy or noisy Sentiment Indicator results meaning the Sentiment Indicator factor model may not be a very stable method to use. 

In [26]:
kappa_vals = [0.1, 0.7, 0.9]
dict_track = {}

- Fama French 3-Factor

In [27]:
#fama_french_3F

note_method = 'FamaFrench3F'
color = 'lightblue'
mu = X.mean(axis=0)
Sigma = fama_french_3F(X_trn)
charts = []

for kappa in kappa_vals:
    w_GMRP_robust, chart = portfolioMaxReturnRobustEllipsoid(mu, Sigma, kappa, color, note_method)
    ref = (note_method, kappa)
    dict_track[ref] = w_GMRP_robust
    charts.append(chart)

display(alt.hconcat(*charts))


alt.HConcatChart(...)

In [28]:
#fama_french_3F with noise

note_method = 'FamaFrench3F'
color = 'lightblue'
mu = X.mean(axis=0)
Sigma = fama_french_3F(X_trn)

for kappa in kappa_vals:
    ref = (note_method, kappa)
    add_noise(mu, Sigma, dict_track[ref], kappa, color, note_method)


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)

- Sentiment Indicator

In [29]:
#Sentiment indicator

note_method = 'SentInd'
color = 'lightgreen'
mu = X.mean(axis=0)
Sigma = sentiment_ind(X_trn)
charts = []

for kappa in kappa_vals:
    w_GMRP_robust, chart = portfolioMaxReturnRobustEllipsoid(mu, Sigma, kappa, color, note_method)
    ref = (note_method, kappa)
    dict_track[ref] = w_GMRP_robust
    charts.append(chart)

display(alt.hconcat(*charts))

alt.HConcatChart(...)

In [30]:
#Sentiment indicator with noise

note_method = 'SentInd'
color = 'lightgreen'
mu = X.mean(axis=0)
Sigma = sentiment_ind(X_trn)

for kappa in kappa_vals:
    ref = (note_method, kappa)
    add_noise(mu, Sigma, dict_track[ref], kappa, color, note_method)


alt.Chart(...)

alt.Chart(...)

alt.Chart(...)